In [1]:
#We install all of the packages necessary
#Sodapy is for Socrata
!pip3 install sodapy
!pip3 install -U pymongo
!pip install -U flask
!pip3 install tomlkit
!pip install -U pyspark

In [2]:
#We import the basic packages to read dataframe, API, Json files, and MongoDB
import pandas as pd
from sodapy import Socrata
import json 
from json import loads
from pymongo import MongoClient
from flask import Flask, request, render_template

In [3]:
#We start and run MongoDB, we call our database TermProject
client1 = MongoClient('localhost',27017)
dbb = client1.TermProject
collection = dbb.test_collection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'TermProject'), 'test_collection')

In [71]:
#This extracts the host server, which is the NYS Governmental Website
client = Socrata("data.ny.gov", None)

Above, we install and import our packages, and run MongoDB. This is the start of our project.

In [70]:
start_date = '2023-03-01'
end_date = '2023-03-31'
rider = client.get("wujg-7c2s", where=f"transit_timestamp >= '{start_date}' AND transit_timestamp <= '{end_date}'",limit=2500000)

Above, we request to retrieve data from NY's dataset identified by the API ID "wujg-7c2s" within March 2023. The request is filtered to include only records with a "transit_timestamp". The limit parameter is set to 2,500,000 rows. This is how we acquire all 4 of our datasets

In [6]:
#Convert into dataframe using PD
ridership = pd.DataFrame.from_records(rider)
ridership[['Date', 'Time']] = ridership['transit_timestamp'].str.split('T', expand=True)

#We split the data and time using str.split, then we extract the day and print a new column that indicates weekday or weekend.
from datetime import datetime
def day_type(date_str):
    day = int(date_str.split('-')[2])
    if (day + 4) % 7 < 5:
        return 1
    else:
        return 2

ridership['day_type'] = ridership['Date'].apply(lambda x: day_type(x))

In [7]:
ridership = ridership[~ridership['station_complex_id'].str.contains('TRAM')]
ridership.station_complex_id.unique()

array(['28', '280', '282', '283', '284', '286', '287', '288', '289',
       '290', '190', '291', '292', '293', '294', '295', '296', '297',
       '298', '299', '3', '30', '300', '301', '303', '304', '305', '306',
       '307', '308', '309', '31', '310', '311', '312', '313', '314',
       '318', '316', '319', '32', '320', '321', '323', '324', '325',
       '326', '327', '328', '329', '33', '333', '334', '153', '336',
       '337', '339', '34', '340', '341', '343', '344', '345', '346',
       '347', '348', '349', '35', '350', '351', '352', '353', '354',
       '355', '356', '357', '358', '359', '36', '360', '361', '362',
       '363', '364', '365', '366', '367', '368', '369', '37', '370',
       '371', '372', '373', '374', '375', '376', '377', '378', '379',
       '38', '380', '381', '382', '383', '384', '385', '386', '387',
       '388', '39', '391', '392', '393', '394', '395', '396', '397',
       '398', '399', '4', '403', '404', '405', '407', '409', '41', '413',
       '414', '416', '

In [8]:
ridership[['latitude','longitude']] = ridership[['latitude','longitude']].astype(float)
ridership.dtypes

transit_timestamp       object
transit_mode            object
station_complex_id      object
station_complex         object
borough                 object
payment_method          object
fare_class_category     object
ridership               object
transfers               object
latitude               float64
longitude              float64
georeference            object
Date                    object
Time                    object
day_type                 int64
dtype: object

These 3 cells above are pre-processing the dataset. We separate time_stampe into Date and Time. We remove any Station_Complex_ID that has 'TRAM' to remove the Roosevelt Island Tram. We convert coordinates into float and day_type into integer. We set the Day_type to 1 (weekday) and 2(weekend) as the rest of the datasets used only use day types.

In [9]:
rider_json = ridership.to_json(orient='records')
rider_json = json.loads(rider_json)
collection = dbb.rider_json
collection.insert_many(rider_json)

print(f"{len(rider_json)} documents inserted into MongoDB successfully.")

2100300 documents inserted into MongoDB successfully.


#### Q2: Which Stations Needs Accesibility Ramps/Elevators (ADA)?

This next Question Helps us understand which stations need ADA (Elevators/Ramps) based on Seniors & Disability Fares.

In [33]:
# Fetch station data from the client's specified API ID
station = client.get("39hk-dx4f", limit=500)
# Convert the station data to a PD
station = pd.DataFrame.from_records(station)
# Remove the last 5 columns from the DataFrame since its TRAMS and Staten Island
station = station.iloc[:, :-5]
# Rename columns 'gtfs_stop_id' to 'stop_id', 'gtfs_latitude' to 'latitude', and 'gtfs_longitude' to 'longitude'
station.rename(columns={'gtfs_stop_id': 'stop_id', 'gtfs_latitude': 'latitude', 'gtfs_longitude': 'longitude'}, inplace=True)
# Create a new column 'station_complex_id' and assign it the values from the 'complex_id' column (similar to ridership)
station['station_complex_id'] = station['complex_id']
# Convert the 'latitude' and 'longitude' columns to float data type
station[['latitude', 'longitude']] = station[['latitude', 'longitude']].astype(float)
station

,stop_id,station_id,complex_id,division,line,stop_name,borough,daytime_routes,structure,latitude,longitude,north_direction_label,south_direction_label,ada,ada_northbound,ada_southbound,station_complex_id
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,1
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,2
2,R04,3,3,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,3
3,R05,4,4,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,4
4,R06,5,5,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,S15,517,517,SIR,Staten Island,Prince's Bay,SI,SIR,Open Cut,40.525507,-74.200064,Ferry,South Shore,0,0,0,517
492,S14,518,518,SIR,Staten Island,Pleasant Plains,SI,SIR,Embankment,40.522410,-74.217847,Ferry,South Shore,0,0,0,518
493,S13,519,519,SIR,Staten Island,Richmond Valley,SI,SIR,Open Cut,40.519631,-74.229141,Ferry,Tottenville,0,0,0,519
494,S09,522,522,SIR,Staten Island,Tottenville,SI,SIR,At Grade,40.512764,-74.251961,Ferry,Last Stop,1,1,1,522


In [34]:
# We inner join the ridership and station dataset based on station complex id
ada = pd.merge(ridership,station[['station_complex_id','ada']], on='station_complex_id', how='inner')
ada.head(5)

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,Date,Time,day_type,ada
0,2023-03-01T00:00:00.000,subway,28,Union St (R),Brooklyn,metrocard,Metrocard - Other,3.0,0.0,40.677315,-73.983109,"{'type': 'Point', 'coordinates': [-73.98310852...",2023-03-01,00:00:00.000,2,0
1,2023-03-01T00:00:00.000,subway,28,Union St (R),Brooklyn,metrocard,Metrocard - Unlimited 7-Day,7.0,0.0,40.677315,-73.983109,"{'type': 'Point', 'coordinates': [-73.98310852...",2023-03-01,00:00:00.000,2,0
2,2023-03-01T00:00:00.000,subway,28,Union St (R),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,2.0,0.0,40.677315,-73.983109,"{'type': 'Point', 'coordinates': [-73.98310852...",2023-03-01,00:00:00.000,2,0
3,2023-03-01T00:00:00.000,subway,28,Union St (R),Brooklyn,omny,OMNY - Full Fare,10.0,0.0,40.677315,-73.983109,"{'type': 'Point', 'coordinates': [-73.98310852...",2023-03-01,00:00:00.000,2,0
4,2023-03-01T01:00:00.000,subway,28,Union St (R),Brooklyn,metrocard,Metrocard - Other,1.0,0.0,40.677315,-73.983109,"{'type': 'Point', 'coordinates': [-73.98310852...",2023-03-01,01:00:00.000,2,0


In [35]:
#We create a new database for this merged dataset on MongoDB
client2 = MongoClient('localhost',27017)
dbb2 = client2.Term
collection2 = dbb2.test_collection
collection2

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Term'), 'test_collection')

In [36]:
#We load the dataset onto MongoDB
ada_json = ada.to_json(orient='records')
ada_json = json.loads(ada_json)
collection2 = dbb2.ada_json
collection2.insert_many(ada_json)

print(f"{len(ada_json)} documents inserted into MongoDB successfully.")

2392692 documents inserted into MongoDB successfully.


In [63]:
q1 = [
    # Match documents where 'ada' field contains '0' (indicating no ADA accessibility)
    {"$match": {"ada": {"$regex": '0'}}},
    # Match documents where the swipe is only by Seniors
    {"$match": {"fare_class_category": {"$regex": 'Seniors'}}},
    # Group documents by 'borough' and calculate the sum of swipes for each group
    {"$group": {"_id": "$borough", "ridership": {"$sum": 1}}},
    # Project the 'borough' and 'ridership' fields
    {"$project": {"borough": "$_id", "ridership": 1, "_id": 0}},
    # Sort by ridership in descending order
    {"$sort": {"ridership": -1}}
]

f1 = collection2.aggregate(q1)

# Print the results
print("Number of Senior Swipes by Borough")
for result in f1:
    print(result)

Number of Senior Swipes by Borough
{'ridership': 231294, 'borough': 'Brooklyn'}
{'ridership': 224574, 'borough': 'Manhattan'}
{'ridership': 99057, 'borough': 'Queens'}
{'ridership': 85302, 'borough': 'Bronx'}
{'ridership': 591, 'borough': 'Staten Island'}


In [38]:
q2 = [
    # Match documents where 'ada' field contains '0' (indicating no ADA accessibility)
    {"$match": {"ada": {"$regex": '0'}}},
    # Match documents where the swipe is only by Seniors
    {"$match": {"fare_class_category": {"$regex": 'Seniors'}}},
    # Group documents by 'station_complex' and calculate the sum of swipes for each group
    {"$group": {"_id": "$station_complex", "ridership": {"$sum": 1}}},
    # Project the 'station_complex' and 'ridership' fields, and exclude the '_id' field
    {"$project": {"station_complex": "$_id", "ridership": 1, "_id": 0}},
    # Sort by ridership in descending order
    {"$sort": {"ridership": -1}}
]

f2 = collection2.aggregate(q2)

# Initialize a count variable to get top 10
count = 0

# Print the results
print("Number of Swipes by Seniors/Disabilities in Stations that do not have Elevators or Ramps")
for result in f2:
    # Print up to the first 10 results
    if count < 10:
        print(result)
        count += 1
    else:
        break

Number of Swipes by Seniors/Disabilities in Stations that do not have Elevators or Ramps
{'ridership': 10503, 'station_complex': '14 St (F,M,1,2,3)/6 Av (L)'}
{'ridership': 9180, 'station_complex': 'Canal St (J,N,Q,R,W,Z,6)'}
{'ridership': 6792, 'station_complex': 'Lexington Av (N,R,W)/59 St (4,5,6)'}
{'ridership': 6276, 'station_complex': 'Bryant Pk (B,D,F,M)/5 Av (7)'}
{'ridership': 6060, 'station_complex': 'Chambers St (A,C)/WTC (E)/Park Pl (2,3)/Cortlandt (R,W)'}
{'ridership': 6036, 'station_complex': 'Delancey St (F)/Essex St (J,M,Z)'}
{'ridership': 5958, 'station_complex': 'Broadway Junction (A,C,J,L,Z)'}
{'ridership': 5268, 'station_complex': '145 St (A,C,B,D)'}
{'ridership': 5088, 'station_complex': '4 Av (F,G)/9 St (R)'}
{'ridership': 4704, 'station_complex': 'Franklin Av (2,3,4,5)/Botanic Garden (S)'}


#### Q3: Which Train Lines perform the best?

This portion uses PySpark to merge and access delays and incidents. The on_time_performance for each line gives us an understanding of which subway lines need improvement.

In [39]:
# We import pyspark and its functions. We then start a SparkSession.
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, DateType, StringType, StructType
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)

24/04/17 11:21:16 WARN Utils: Your hostname, Isais-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.206.37.35 instead (on interface en0)
24/04/17 11:21:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/17 11:21:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Apache Spark Version 3.5.1


In [40]:
# Get the specified API ID to import the Subway Performance Dataset, convert to PD
delay = client.get("vtvh-gimj",limit = 3000)
delay = pd.DataFrame.from_records(delay)
#Convert performance into float, round to 4 decimals
delay['terminal_on_time_performance'] = delay['terminal_on_time_performance'].astype(float)
delay['terminal_on_time_performance'] = delay['terminal_on_time_performance'].round(4)
delay

,month,division,line,day_type,num_on_time_trips,num_sched_trips,terminal_on_time_performance
0,2024-02,A DIVISION,1,1,7433,9176,0.8100
1,2024-02,A DIVISION,1,2,2650,3492,0.7589
2,2024-02,A DIVISION,2,1,4687,6534,0.7173
3,2024-02,A DIVISION,2,2,653,2106,0.3101
4,2024-02,A DIVISION,3,2,648,1050,0.6171
...,...,...,...,...,...,...,...
2331,2020-01,B DIVISION,S Fkln,2,1889,1894,0.9974
2332,2020-01,B DIVISION,S Rock,1,3596,3696,0.9729
2333,2020-01,B DIVISION,S Rock,2,1360,1395,0.9749
2334,2020-01,Systemwide,Systemwide,1,150643,180963,0.8325


In [41]:
# Get the specified API ID to import the NYC Subway incidents Dataset, convert to PD
incidents = client.get("j6d2-s8m2", limit=2000)
incidents = pd.DataFrame.from_records(incidents)
incidents

,month,division,line,day_type,category,count
0,2024-02,A DIVISION,1,1,Other,1
1,2024-02,A DIVISION,1,2,Subway Car,1
2,2024-02,A DIVISION,2,1,Persons on Trackbed/Police/Medical,1
3,2024-02,A DIVISION,2,1,Track,1
4,2024-02,A DIVISION,2,1,Signals,1
...,...,...,...,...,...,...
1984,2020-01,Systemwide,Systemwide,1,Subway Car,2
1985,2020-01,Systemwide,Systemwide,1,Track,4
1986,2020-01,Systemwide,Systemwide,2,Other,3
1987,2020-01,Systemwide,Systemwide,1,Other,5


In [42]:
# Checking to see which are the different categories
incidents.category.unique()

array(['Other', 'Subway Car', 'Persons on Trackbed/Police/Medical',
       'Track', 'Signals', 'Stations and Structure'], dtype=object)

In [43]:
# Create Spark DF from the delay and incidents datasets
delay_spark = spark.createDataFrame(delay)
incidents_spark = spark.createDataFrame(incidents)

# Join the delay and incidents DataFrames on common columns
delay_inc = delay_spark.join(incidents_spark, on=["month", "division", "line", "day_type"], how="inner")

In [44]:
# Show the total count for each incidental category
incident_counts = delay_inc.groupBy("category").count()
incident_counts.show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|               Track|  313|
|          Subway Car|  220|
|             Signals|  501|
|Persons on Trackb...|  555|
|               Other|  210|
|Stations and Stru...|  189|
+--------------------+-----+



This shows the Ontime ratings and Safety Performance. Please note it is the same metric.

In [48]:
# Group delay incidents by category and calculate the average performance
incident_performance = delay_inc.groupBy("category") \
    .agg(F.avg("terminal_on_time_performance").alias("average_performance"))

# Round the average_performance column to 4 decimal places using F
incident_performance = incident_performance.withColumn("average_performance", F.round("average_performance", 4))

# Print the average on-time rating by each incident category in descending order
print("Average On-time Rating by each Incident")
incident_performance.orderBy(F.desc("average_performance")).show()

Average Ontime Rating by each Incident


+--------------------+-------------------+
|            category|average_performance|
+--------------------+-------------------+
|          Subway Car|             0.8083|
|               Other|             0.8069|
|Persons on Trackb...|             0.8061|
|             Signals|             0.8048|
|Stations and Stru...|             0.7993|
|               Track|             0.7945|
+--------------------+-------------------+



In [50]:
# Filter delay incidents by excluding "Persons" and "Other" categories, as well as Shuttles
average_performance_by_line = delay_inc \
    .where(~(F.col("category").isin(["Persons", "Other"]))) \
    .where(~(F.col("line").isin(["S 42nd", "S Fkln","Systemwide"]))) \
    .groupBy("line") \
    .agg(F.avg("terminal_on_time_performance").alias("average_performance")) \
    .orderBy(F.desc("average_performance"))

# Round the average_performance column to 4 decimal places
average_performance_by_line = average_performance_by_line.withColumn("average_performance", F.round("average_performance", 4))

# Print the average on-time rating by subway line, excluding Shuttles
print("Average On-time Rating by Subway Line")
print("(Excluding 'Persons on Trackbed' and 'Other' Categories as well as Shuttles and Systemwide incidents)")
average_performance_by_line.orderBy(F.desc("average_performance")).show()
print("J and Z lines run on the same line, have the same stops and terminal stations")


Average Ontime Rating by Subway Line
(Excluding 'Persons on Trackbed' and 'Other' Categories as well as Shuttles and Systemwide incidents)


+----+-------------------+
|line|average_performance|
+----+-------------------+
|   L|             0.9139|
|   7|             0.9029|
|   G|             0.8804|
|  JZ|             0.8491|
|   1|             0.8418|
|   3|             0.8342|
|   5|             0.8277|
|   6|             0.8239|
|   M|              0.822|
|   4|             0.8208|
|   R|             0.8053|
|   Q|             0.7561|
|   D|              0.755|
|   E|              0.749|
|   2|             0.7485|
|   N|             0.7429|
|   B|             0.7302|
|   C|             0.7293|
|   A|             0.7228|
|   F|             0.7099|
+----+-------------------+

J and Z lines runs on the same line, have the same stops and terminal stations


In [54]:
# Filter delay incidents by "Persons on Trackbed/Police/Medical" category and exclude Shuttles
average_performance_person = delay_inc \
    .where(F.col("category") == "Persons on Trackbed/Police/Medical") \
    .where(~(F.col("line").isin(["S 42nd", "S Fkln","Systemwide"]))) \
    .groupBy("line") \
    .agg(F.avg("terminal_on_time_performance").alias("average_safety")) \
    .orderBy(F.asc("average_safety")) \
    .limit(10)

# Round the average_safety column to 4 decimal places
average_performance_person = average_performance_person.withColumn("average_safety", F.round("average_safety", 4))

# Print the Top 10 worst safety rating by subway line, including only "Persons on Trackbed" category and excluding Shuttles
print("Worst Safety Rating by Subway Line")
print("(Only Includes 'Persons on Trackbed'. Excludes Shuttles and Systemwide incidents)")
average_performance_person.orderBy(F.asc("average_safety")).show()
print("Ontime Performance is used to judge how less safe a train line.")
print("The lower the performance, the more likely incidents occur.")

Worst Safety Rating by Subway Line
(Only Includes 'Persons on Trackbed'. Excludes Shuttles and Systemwide incidents)
+----+--------------+
|line|average_safety|
+----+--------------+
|   F|         0.706|
|   A|        0.7122|
|   N|        0.7346|
|   B|        0.7355|
|   C|        0.7498|
|   2|        0.7559|
|   E|        0.7605|
|   D|        0.7659|
|   Q|         0.766|
|   R|        0.8116|
+----+--------------+

J and Z lines runs on the same line, have the same stops and terminal stations


In [60]:
# Filter delay incidents excluding "Persons" and "Other" categories and Shuttles
average_performance = delay_inc \
    .where(~(F.col("category").isin(["Persons", "Other"]))) \
    .where(~(F.col("line").isin(["S 42nd", "S Fkln", "Systemwide"]))) \
    .groupBy("line") \
    .pivot("day_type") \
    .agg(F.avg("terminal_on_time_performance")) \
    .withColumnRenamed("1", "weekday") \
    .withColumnRenamed("2", "weekend") \
    .orderBy(F.desc("weekday"))

# Round the weekday and weekend columns to 4 decimal places
average_performance = average_performance \
    .withColumn("weekday", F.round("weekday", 4)) \
    .withColumn("weekend", F.round("weekend", 4))

# Print the average on-time rating by subway line by day type
print("Average On-time Rating by Subway Line by Day Type")
print("(Excluding 'Persons on Trackbed' and 'Other' Categories as well as Shuttles and Systemwide incidents)")
average_performance.show()
print("Note: J and Z lines run on the same line, have the same stops and terminal stations")
print("Note: B trains only run on Weekdays 6AM - 12AM")

Average On-time Rating by Subway Line by Day Type
(Excluding 'Persons on Trackbed' and 'Other' Categories as well as Shuttles and Systemwide incidents)
+----+-------+-------+
|line|weekday|weekend|
+----+-------+-------+
|   L| 0.9134|  0.915|
|   7| 0.9056| 0.8753|
|   G| 0.8694| 0.8937|
|   3| 0.8587| 0.7457|
|  JZ| 0.8561| 0.8247|
|   1| 0.8395| 0.8525|
|   4| 0.8325| 0.7686|
|   5|  0.829| 0.7269|
|   6| 0.8238| 0.8239|
|   M| 0.8188| 0.8796|
|   R| 0.7998| 0.8346|
|   2| 0.7667| 0.6329|
|   Q|  0.766| 0.7147|
|   C| 0.7648| 0.6481|
|   N|  0.747| 0.7074|
|   D|  0.747| 0.7889|
|   E| 0.7432| 0.8139|
|   B| 0.7302|   NULL|
|   A| 0.7174| 0.7475|
|   F| 0.7084| 0.7291|
+----+-------+-------+

Note: J and Z lines run on the same line, have the same stops and terminal stations
Note: B trains only run on Weekdays 6AM - 12AM


In [61]:
# Filter delay incidents for "Persons on Trackbed/Police/Medical" category and exclude Shuttles
average_performance = delay_inc \
    .where(F.col("category") == "Persons on Trackbed/Police/Medical") \
    .where(~(F.col("line").isin(["S 42nd", "S Fkln", "Systemwide"]))) \
    .groupBy("line") \
    .pivot("day_type") \
    .agg(F.avg("terminal_on_time_performance")) \
    .withColumnRenamed("1", "weekday") \
    .withColumnRenamed("2", "weekend") \
    .orderBy(F.asc("weekday")) \
    .limit(10)

# Round the weekday and weekend columns to 4 decimal places
average_performance = average_performance \
    .withColumn("weekday", F.round("weekday", 4)) \
    .withColumn("weekend", F.round("weekend", 4))

# Print the Top 10 worst safety rating by subway line, including only "Persons on Trackbed" category and excluding Shuttles
print("Worst Safety Rating by Subway Line by Day Type")
print("(Only Includes 'Persons on Trackbed'. Excludes Shuttles and Systemwide incidents)")
average_performance.show()
print("Ontime Performance is used to judge how less safe a train line.")
print("The lower the performance, the more likely incidents occur.")
print("Note: J and Z lines run on the same line, have the same stops and terminal stations")
print("Note: B trains do not run on Weekdays 9:30PM - 6AM and on Weekends")

Worst Safety Rating by Subway Line by Day Type
(Only Includes 'Persons on Trackbed'. Excludes Shuttles and Systemwide incidents)


+----+-------+-------+
|line|weekday|weekend|
+----+-------+-------+
|   A| 0.7033| 0.7344|
|   F|  0.706| 0.7061|
|   B| 0.7355|   NULL|
|   N| 0.7377| 0.7123|
|   E| 0.7542| 0.8396|
|   D| 0.7603| 0.8023|
|   2| 0.7708| 0.6724|
|   Q| 0.7775| 0.7338|
|   C| 0.7859| 0.6415|
|   R| 0.8039| 0.8345|
+----+-------+-------+

Note: J and Z lines run on the same line, have the same stops and terminal stations
Note: B trains do not run on Weekdays 9:30PM - 6AM and on Weekends
